# Segmenting and clustering neighborhoods in city toronto 

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Download and explore dataset

In [6]:
#import the library used to query a website
import urllib.request #if you are using python3+ version, import urllib.request

In [7]:
wiki ="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

### Query the website and return the html to the variable 'page'

In [8]:
#Query the website and return the html to the variable 'page'
page = urllib.request.urlopen(wiki) #For python 3 use urllib.request.urlopen(wiki)

### import the Beautiful soup functions to parse the data returned from the website

In [9]:
#import the Beautiful soup functions to parse the data returned from the website
from bs4 import BeautifulSoup

### Parse the html in the 'page' variable, and store it in Beautiful Soup format

In [10]:
#Parse the html in the 'page' variable, and store it in Beautiful Soup format
soup = BeautifulSoup(page)
#print(soup)

### Extract Table

In [11]:
all_tables=soup.find_all('table')
#print(all_tables)

### Find the right table

In [12]:
right_table=soup.find('table', class_='wikitable sortable')
#print(right_table)

### Extract the information to DataFrame

In [13]:
#Generate lists
A=[]
B=[]
d=[]
for row in right_table.findAll("tr"):
    cells = row.findAll('td')
    if len(cells)==3: #Only extract table body not heading
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        d.append(cells[2].find(text=True))
#print(A)
#print(B)
#print(d)

In [14]:
for n, i in enumerate(B):
    if i == 'Not assigned':
        B[n] = 'NaN'
#print(B)

In [15]:
for n, i in enumerate(d):
    if i == 'Not assigned\n':
        d[n] = 'NaN'
#print(d)

In [16]:
C = []
for i in d:
    C.append(i.strip())
#C

In [17]:
l = {'Postcode': A, 'Borough': B, 'Neighbourhood': d}
df = pd.DataFrame(data=l)
df

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,NaN
9,M8A,NaN,NaN


### Ignore cells with a borough that is Not assigned

In [18]:
df = df[df.Borough != 'NaN']
df.head(7)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,NaN


In [19]:
# reset index
df.reset_index(drop=True, inplace=True)
df.head(8)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,NaN
7,M9A,Etobicoke,Islington Avenue


### replace the neighbourhood with not assigned same as borough


In [21]:
import numpy as np
df['Neighbourhood'].replace('NaN', df['Borough'],inplace=True)
df.head(8)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue


### group neighbourhood for same postcode area

In [22]:
df = df.groupby('Postcode').agg({'Borough':'first','Neighbourhood': ', '.join}).reset_index()
df = df.replace('\n','', regex=True)
df.head(8)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"


In [23]:
df.shape

(103, 3)

### adding two more columns to our dataframe i.e. latitude and longitude

In [24]:

df['Latitude']= ""
df['Longitude']= ""
df.head(9)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",,
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",,
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",,
3,M1G,Scarborough,Woburn,,
4,M1H,Scarborough,Cedarbrae,,
5,M1J,Scarborough,Scarborough Village,,
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",,
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",,
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",,


In [25]:
#!conda install -c conda-forge geopy --yes

In [26]:
CLIENT_ID = 'CIFRIO2D1HYSA2CSIZ3PX32Y34CHCSIHBDVQJMTKRNHRDYVN'# your Foursquare ID
CLIENT_SECRET = '2NV43JJEN2DCNZRFU1TYZLASKAPLXAFBUXB3XCOL04AY4LZU' # your Foursquare id
VERSION = '20180604'
LIMIT = 30

### calling geolocator to find latitude and longitude

In [27]:
for index, row in df.iterrows():
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(row['Borough'])
    row['Latitude'] = location.latitude
    row['Longitude'] = location.longitude

### csv file to create the dataframe

In [28]:
df.to_csv('toronto.csv')

In [29]:
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",54.2848,-0.409034
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",54.2848,-0.409034
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",54.2848,-0.409034
3,M1G,Scarborough,Woburn,54.2848,-0.409034
4,M1H,Scarborough,Cedarbrae,54.2848,-0.409034
5,M1J,Scarborough,Scarborough Village,54.2848,-0.409034
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",54.2848,-0.409034
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",54.2848,-0.409034
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",54.2848,-0.409034
9,M1N,Scarborough,"Birch Cliff, Cliffside West",54.2848,-0.409034


In [30]:
import folium

### Analyzing neighbourhoods of toronto

In [31]:
print('The dataframe has {} Boroughs and {} Neighbourhood.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 Boroughs and 103 Neighbourhood.


### Use geopy library to get the latitude and longitude values of New York City.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [32]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [65]:
# Create a map of Toronto with neighborhoods superimposed on top.

In [33]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

# Let's define a function to get nearby venues of a borough

In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


## Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [35]:
Toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

### Let's check the size of the resulting dataframe

In [36]:
print(Toronto_venues.shape)
Toronto_venues.head()

(2277, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",54.28476,-0.409034,Gianni's,54.283031,-0.405256,Italian Restaurant
1,"Rouge, Malvern",54.28476,-0.409034,Eat Me Cafe,54.280697,-0.406778,Restaurant
2,"Rouge, Malvern",54.28476,-0.409034,Aldi,54.281965,-0.407199,Supermarket
3,"Rouge, Malvern",54.28476,-0.409034,Domino's Pizza,54.283844,-0.403644,Pizza Place
4,"Rouge, Malvern",54.28476,-0.409034,Pizza Hut,54.280872,-0.407803,Pizza Place


### Let's check how many venues were returned for each neighborhood

In [37]:
Toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",30,30,30,30,30,30
Agincourt,6,6,6,6,6,6
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",6,6,6,6,6,6
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",5,5,5,5,5,5
"Alderwood, Long Branch",5,5,5,5,5,5
"Bathurst Manor, Downsview North, Wilson Heights",30,30,30,30,30,30
Bayview Village,30,30,30,30,30,30
"Bedford Park, Lawrence Manor East",30,30,30,30,30,30
Berczy Park,30,30,30,30,30,30


### Let's find out how many unique categories can be curated from all the returned venues

In [38]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 89 uniques categories.


# Analyze Each Neighborhood

In [39]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighbourhood,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Athletics & Sports,Bakery,Bank,Bar,Beer Bar,Beer Store,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Electronics Store,Farmers Market,French Restaurant,Furniture / Home Store,Garden,Gastropub,Grocery Store,Historic Site,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lingerie Store,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Music Venue,Neighborhood,Office,Opera House,Park,Performing Arts Venue,Pet Store,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Smoke Shop,Smoothie Shop,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Theater,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### And let's examine the new dataframe size.

In [40]:
Toronto_onehot.shape

(2277, 90)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [41]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped

,Neighbourhood,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Athletics & Sports,Bakery,Bank,Bar,Beer Bar,Beer Store,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Electronics Store,Farmers Market,French Restaurant,Furniture / Home Store,Garden,Gastropub,Grocery Store,Historic Site,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Lingerie Store,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Music Venue,Neighborhood,Office,Opera House,Park,Performing Arts Venue,Pet Store,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Smoke Shop,Smoothie Shop,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Theater,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Adelaide, King, Richmond",0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.033333,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.0,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.066667,0.000000,0.033333,0.000000,0.000000,0.000000
1,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

### Let's confirm the new size

In [42]:
Toronto_grouped.shape

(103, 90)

### Let's print each neighborhood along with the top 5 most common venues

In [43]:
num_top_venues = 5

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0       Clothing Store  0.10
1  American Restaurant  0.07
2                Plaza  0.07
3              Theater  0.07
4           Restaurant  0.07


----Agincourt----
                venue  freq
0         Pizza Place  0.33
1  Italian Restaurant  0.17
2          Restaurant  0.17
3       Grocery Store  0.17
4         Supermarket  0.17


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0         Pizza Place  0.33
1  Italian Restaurant  0.17
2          Restaurant  0.17
3       Grocery Store  0.17
4         Supermarket  0.17


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
         venue  freq
0   Playground   0.4
1       Garden   0.2
2  Coffee Shop   0.2
3  Supermarket   0.2
4       Office   0.0


----Alderwood, Long Branch----
         venue  freq
0   Playground   0.4
1       Garden   0.2
2  Coffee Shop   0.2
3  Supe

### Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.

In [63]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [64]:
import numpy as np
import pandas as pd
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Clothing Store,American Restaurant,Theater,Cosmetics Shop,Restaurant,Plaza,Tea Room,Shopping Mall,Music Venue,Movie Theater
1,Agincourt,Pizza Place,Italian Restaurant,Grocery Store,Restaurant,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Pizza Place,Italian Restaurant,Grocery Store,Restaurant,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Playground,Coffee Shop,Garden,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
4,"Alderwood, Long Branch",Playground,Coffee Shop,Garden,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


# Cluster Neighborhoods

In [65]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 2, 1, 1, 3, 3, 3, 0, 2], dtype=int32)

#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [66]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",54.2848,-0.409034,2,Pizza Place,Italian Restaurant,Grocery Store,Restaurant,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",54.2848,-0.409034,2,Pizza Place,Italian Restaurant,Grocery Store,Restaurant,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",54.2848,-0.409034,2,Pizza Place,Italian Restaurant,Grocery Store,Restaurant,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store
3,M1G,Scarborough,Woburn,54.2848,-0.409034,2,Pizza Place,Italian Restaurant,Grocery Store,Restaurant,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store
4,M1H,Scarborough,Cedarbrae,54.2848,-0.409034,2,Pizza Place,Italian Restaurant,Grocery Store,Restaurant,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store


### Finally, let's visualize the resulting clusters

In [67]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters
Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

## Cluster 1

In [68]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,0,Clothing Store,American Restaurant,Theater,Cosmetics Shop,Restaurant,Plaza,Tea Room,Shopping Mall,Music Venue,Movie Theater
51,Downtown Toronto,0,Clothing Store,American Restaurant,Theater,Cosmetics Shop,Restaurant,Plaza,Tea Room,Shopping Mall,Music Venue,Movie Theater
52,Downtown Toronto,0,Clothing Store,American Restaurant,Theater,Cosmetics Shop,Restaurant,Plaza,Tea Room,Shopping Mall,Music Venue,Movie Theater
53,Downtown Toronto,0,Clothing Store,American Restaurant,Theater,Cosmetics Shop,Restaurant,Plaza,Tea Room,Shopping Mall,Music Venue,Movie Theater
54,Downtown Toronto,0,Clothing Store,American Restaurant,Theater,Cosmetics Shop,Restaurant,Plaza,Tea Room,Shopping Mall,Music Venue,Movie Theater
55,Downtown Toronto,0,Clothing Store,American Restaurant,Theater,Cosmetics Shop,Restaurant,Plaza,Tea Room,Shopping Mall,Music Venue,Movie Theater
56,Downtown Toronto,0,Clothing Store,American Restaurant,Theater,Cosmetics Shop,Restaurant,Plaza,Tea Room,Shopping Mall,Music Venue,Movie Theater
57,Downtown Toronto,0,Clothing Store,American Restaurant,Theater,Cosmetics Shop,Restaurant,Plaza,Tea Room,Shopping Mall,Music Venue,Movie Theater
58,Downtown Toronto,0,Clothing Store,American Restaurant,Theater,Cosmetics Shop,Restaurant,Plaza,Tea Room,Shopping Mall,Music Venue,Movie Theater
59,Downtown Toronto,0,Clothing Store,American Restaurant,Theater,Cosmetics Shop,Restaurant,Plaza,Tea Room,Shopping Mall,Music Venue,Movie Theater


In [69]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
88,Etobicoke,1,Playground,Coffee Shop,Garden,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
89,Etobicoke,1,Playground,Coffee Shop,Garden,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
90,Etobicoke,1,Playground,Coffee Shop,Garden,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
91,Etobicoke,1,Playground,Coffee Shop,Garden,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
92,Etobicoke,1,Playground,Coffee Shop,Garden,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
93,Etobicoke,1,Playground,Coffee Shop,Garden,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
94,Etobicoke,1,Playground,Coffee Shop,Garden,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
95,Etobicoke,1,Playground,Coffee Shop,Garden,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
99,Etobicoke,1,Playground,Coffee Shop,Garden,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
100,Etobicoke,1,Playground,Coffee Shop,Garden,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


In [70]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2,Pizza Place,Italian Restaurant,Grocery Store,Restaurant,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store
1,Scarborough,2,Pizza Place,Italian Restaurant,Grocery Store,Restaurant,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store
2,Scarborough,2,Pizza Place,Italian Restaurant,Grocery Store,Restaurant,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store
3,Scarborough,2,Pizza Place,Italian Restaurant,Grocery Store,Restaurant,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store
4,Scarborough,2,Pizza Place,Italian Restaurant,Grocery Store,Restaurant,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store
5,Scarborough,2,Pizza Place,Italian Restaurant,Grocery Store,Restaurant,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store
6,Scarborough,2,Pizza Place,Italian Restaurant,Grocery Store,Restaurant,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store
7,Scarborough,2,Pizza Place,Italian Restaurant,Grocery Store,Restaurant,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store
8,Scarborough,2,Pizza Place,Italian Restaurant,Grocery Store,Restaurant,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store
9,Scarborough,2,Pizza Place,Italian Restaurant,Grocery Store,Restaurant,Supermarket,Donut Shop,Concert Hall,Cosmetics Shop,Deli / Bodega,Department Store


In [71]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,North York,3,Ramen Restaurant,Coffee Shop,Café,Korean Restaurant,Restaurant,Japanese Restaurant,Pet Store,Burrito Place,Plaza,Pool
18,North York,3,Ramen Restaurant,Coffee Shop,Café,Korean Restaurant,Restaurant,Japanese Restaurant,Pet Store,Burrito Place,Plaza,Pool
19,North York,3,Ramen Restaurant,Coffee Shop,Café,Korean Restaurant,Restaurant,Japanese Restaurant,Pet Store,Burrito Place,Plaza,Pool
20,North York,3,Ramen Restaurant,Coffee Shop,Café,Korean Restaurant,Restaurant,Japanese Restaurant,Pet Store,Burrito Place,Plaza,Pool
21,North York,3,Ramen Restaurant,Coffee Shop,Café,Korean Restaurant,Restaurant,Japanese Restaurant,Pet Store,Burrito Place,Plaza,Pool
22,North York,3,Ramen Restaurant,Coffee Shop,Café,Korean Restaurant,Restaurant,Japanese Restaurant,Pet Store,Burrito Place,Plaza,Pool
23,North York,3,Ramen Restaurant,Coffee Shop,Café,Korean Restaurant,Restaurant,Japanese Restaurant,Pet Store,Burrito Place,Plaza,Pool
24,North York,3,Ramen Restaurant,Coffee Shop,Café,Korean Restaurant,Restaurant,Japanese Restaurant,Pet Store,Burrito Place,Plaza,Pool
25,North York,3,Ramen Restaurant,Coffee Shop,Café,Korean Restaurant,Restaurant,Japanese Restaurant,Pet Store,Burrito Place,Plaza,Pool
26,North York,3,Ramen Restaurant,Coffee Shop,Café,Korean Restaurant,Restaurant,Japanese Restaurant,Pet Store,Burrito Place,Plaza,Pool


In [72]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,4,Chinese Restaurant,Japanese Restaurant,Poke Place,Park,Opera House,Neighborhood,Miscellaneous Shop,Middle Eastern Restaurant,Concert Hall,Coffee Shop
41,East Toronto,4,Chinese Restaurant,Japanese Restaurant,Poke Place,Park,Opera House,Neighborhood,Miscellaneous Shop,Middle Eastern Restaurant,Concert Hall,Coffee Shop
42,East Toronto,4,Chinese Restaurant,Japanese Restaurant,Poke Place,Park,Opera House,Neighborhood,Miscellaneous Shop,Middle Eastern Restaurant,Concert Hall,Coffee Shop
43,East Toronto,4,Chinese Restaurant,Japanese Restaurant,Poke Place,Park,Opera House,Neighborhood,Miscellaneous Shop,Middle Eastern Restaurant,Concert Hall,Coffee Shop
44,Central Toronto,4,Chinese Restaurant,Japanese Restaurant,Poke Place,Park,Opera House,Neighborhood,Miscellaneous Shop,Middle Eastern Restaurant,Concert Hall,Coffee Shop
45,Central Toronto,4,Chinese Restaurant,Japanese Restaurant,Poke Place,Park,Opera House,Neighborhood,Miscellaneous Shop,Middle Eastern Restaurant,Concert Hall,Coffee Shop
46,Central Toronto,4,Chinese Restaurant,Japanese Restaurant,Poke Place,Park,Opera House,Neighborhood,Miscellaneous Shop,Middle Eastern Restaurant,Concert Hall,Coffee Shop
47,Central Toronto,4,Chinese Restaurant,Japanese Restaurant,Poke Place,Park,Opera House,Neighborhood,Miscellaneous Shop,Middle Eastern Restaurant,Concert Hall,Coffee Shop
48,Central Toronto,4,Chinese Restaurant,Japanese Restaurant,Poke Place,Park,Opera House,Neighborhood,Miscellaneous Shop,Middle Eastern Restaurant,Concert Hall,Coffee Shop
49,Central Toronto,4,Chinese Restaurant,Japanese Restaurant,Poke Place,Park,Opera House,Neighborhood,Miscellaneous Shop,Middle Eastern Restaurant,Concert Hall,Coffee Shop
